<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Callbacks-as-GUI-events" data-toc-modified-id="Callbacks-as-GUI-events-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Callbacks as GUI events</a></span><ul class="toc-item"><li><span><a href="#Creating-our-own-callback" data-toc-modified-id="Creating-our-own-callback-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Creating our own callback</a></span></li><li><span><a href="#Classes-(not-functions)-as-callbacks" data-toc-modified-id="Classes-(not-functions)-as-callbacks-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Classes (not functions) as callbacks</a></span></li><li><span><a href="#Multiple-callbacks;-*args-and-**kwargs" data-toc-modified-id="Multiple-callbacks;-*args-and-**kwargs-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Multiple callbacks; <code>*args</code> and <code>**kwargs</code></a></span></li><li><span><a href="#Using-callbacks-to-modify-behavior" data-toc-modified-id="Using-callbacks-to-modify-behavior-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Using callbacks to modify behavior</a></span></li></ul></li></ul></div>

From https://www.youtube.com/watch?v=HR0lt1hlR6U&feature=youtu.be&list=PLfYUBJiXbdtTIdtE1U8qgyxo4Jy2Y91uj&t=445

In [ ]:
import torch
import matplotlib.pyplot as plt

# Callbacks as GUI events

In [ ]:
import ipywidgets as widgets

In [ ]:
def f(o): print('hi')

"The button widget is used tohandle mouse clicks. The on_click method of the Button can be used to register a function to be called when the button is clicked."

In [ ]:
w = widgets.Button(description='Click me')

In [ ]:
w

Button(description='Click me', style=ButtonStyle())

hi
hi
hi


In [ ]:
w.on_click(f)

When callbacks are used this way, they are often called "events". What we did is we told w that when a click occurs, we should "call back" to the f function and run it. `f` here is a callback (a callback is a concept). It's a function. Callbacks are functions. (Callbacks are a kind of "function pointer").

## Creating our own callback

The event this function will call back on is "after a calculation is complete."

In [ ]:
from time import sleep

In [ ]:
# this is a toy example that does 5 calculations.
# each calculation can be thought of as an epoch.
def train_n_sessions(n=5):
    res = 0
    for i in range(n):
        res += i*i
        sleep(1)
    return res

In [ ]:
train_n_sessions()

30

What I, Rory, really want to know is: how is this thing going? So, let's modify this example to take a callback!

In [ ]:
# this is a toy example that does 5 calculations.
# each calculation can be thought of as an epoch.
def train_n_sessions(n=5, cb=None):
    res = 0
    for i in range(n):
        res += i*i
        sleep(1)
        if cb: cb(i) # if there's a callback, call it and pass in epoch number.
    return res

In [ ]:
def show_progress(epoch): print(f"Finished epoch {epoch}.")

In [ ]:
train_n_sessions()

30

In [ ]:
train_n_sessions(cb=show_progress)

Finished epoch 0.
Finished epoch 1.
Finished epoch 2.
Finished epoch 3.
Finished epoch 4.


30

Awesome! My first callback :) . Tip: a call-back is also known as a "call-after" function, and is defined as any function that is passed as an argument to other code. That other code is expected to "call back" to that function at a specified time. (Callable classes can also be used as callbacks).

Lambdas work.

In [ ]:
train_n_sessions(cb=lambda x: print(f"finished epoch {x}"))

finished epoch 0
finished epoch 1
finished epoch 2
finished epoch 3
finished epoch 4


30

What if we wanted to pass an argument to the callback, such as a phrase to state before the epoch? Let's try.

In [ ]:
def show_progress(phrase,epoch): print(f"{phrase} {epoch}.")

In [ ]:
train_n_sessions(cb=lambda x: show_progress("Yip-yip!",x))

Yip-yip! 0.
Yip-yip! 1.
Yip-yip! 2.
Yip-yip! 3.
Yip-yip! 4.


30

This is a function that returns a function.

In [ ]:
def make_show_progress(phrase):
    def _inner(epoch): print(f"{phrase} {epoch}")
    return _inner

In [ ]:
show_prog = make_show_progress("Epoch completed:")

In [ ]:
train_n_sessions(cb=show_prog)

Epoch completed: 0
Epoch completed: 1
Epoch completed: 2
Epoch completed: 3
Epoch completed: 4


30

*Side note:* Congrats, Rory, on making your first **closure**!! A function that stores information from the external context and the information can be different every time is called a closure. Google: "A closure is a record storing a function together with an environment." From stack overflow: "A closure is a persistent scope which holds on to local variables even after the code execution has moved out of that block. Langs that support closures allow you to keep a reference to a scope. The scope object and all of its local variables are tied to the function and will persist as long as that function persists. This gives function portability."

The implementation of a closure often looks like a function `f` that returns a function `g`, s.t. the function g is defined with variables that are local to f. Here's what that looks like:

`def f():
    local_var = 1
    def g(x): return x + local_var
    return g`
    
Here's a silly example:

`def make_greeting():
    joke_of_the_day = get_joke()
    def greeting(name):
        print(f"Hi {name}! {joke_of_the_day}.")
    return greeting`
    
In a lang that doesn't support closures, joke_of_the_day would be garbage collected and tossed out when make_greeting is exited. But since python supports closures, it persists because the variable scope is created when the function is first declared and persists as long as the function does.

In [ ]:
from functools import partial

Because it's so common to want to turn a function of many params into a function of one param, python has a builtin for it called partial. This is great for fixing multiple args in a function.

In [ ]:
train_two_sessions = partial(train_n_sessions,n=2)

In [ ]:
train_two_sessions(cb=show_prog)

Epoch completed: 0
Epoch completed: 1


1

It apparantly works by setting defaults for the function's args; you can still pass in a value for n in train_two_sessions.

In [ ]:
train_two_sessions(n=1,cb=show_prog)

Epoch completed: 0


0

## Classes (not functions) as callbacks

Pretty much any place you can use a closure, you can use a class. The state can be stored in `__init__`, and `__call__` will hold the callable. (This is called a "functor" in other langs, or a "callable" in python).

In [ ]:
class ProgressShowingCallback():
    def __init__(self, phrase='Finished epoch'): self.phrase = phrase
    def __call__(self, epoch): print(f"{self.phrase} {epoch}")

In [ ]:
cb_cls = ProgressShowingCallback()

In [ ]:
cb_cls(1)

Finished epoch 1


In [ ]:
train_n_sessions(cb=cb_cls)

Finished epoch 0
Finished epoch 1
Finished epoch 2
Finished epoch 3
Finished epoch 4


30

## Multiple callbacks; `*args` and `**kwargs`

In [ ]:
def f(*args, **kwargs): print(f"args: {args} | kwargs: {kwargs}")

In [ ]:
f(1,2,3,'rice is nice',x=42,y=69,z='blink182')

args: (1, 2, 3, 'rice is nice') | kwargs: {'x': 42, 'y': 69, 'z': 'blink182'}


PyTorch's `nn.Sequential` takes in argumetns as `*args`.

A common way to use this is if you want to wrap an object to create a new object, you can define the new obj's args, and then use kwargs to pass off all the stuff you'd normally pass on to the wrapped object. Note that the new object will NOT show the old object's args when you tab-search or tab-complete, so the full arg list will be hidden from end users. Basically, the end user will have to already know what the wrapped object's args are to use the new object.

Callbacks are a place where you really want to use kwargs. Here I have two callbacks: cb.before_epoch and cb.after_epoch. I can't use cb(i) anymore; I have to use these methods.

In [ ]:
def do_calc(x): sleep(1); return x*x

In [ ]:
def train_n_sessions(n=5, cb=None):
    res = 0
    for i in range(n):
        # Before epoch
        if cb: cb.before_epoch(i)
        calc = do_calc(i)
        res += calc
        # After epoch
        if cb: cb.after_epoch(i, val=res)
    return res

In [ ]:
class PrintStepCallback():
    def __init__(self): pass
    def before_epoch(self, *args, **kwargs): print(f"Epoch starting")
    def after_epoch (self, *args, **kwargs): print(f"Epoch finished")

In [ ]:
cb_cls = PrintStepCallback()

In [ ]:
train_n_sessions(cb=cb_cls)

Epoch starting
Epoch finished
Epoch starting
Epoch finished
Epoch starting
Epoch finished
Epoch starting
Epoch finished
Epoch starting
Epoch finished


30

In the above, we used args and kwargs to create functions (methods) that don't care about one or more parameters; they're more flexible. "If you put in both positional and keyword arguments, it will always work everywhere."

Let's actually use the values passed now.

In [ ]:
class PrintStepCallback():
    def __init__(self): pass
    def before_epoch(self, epoch, **kwargs): print(f"Starting epoch {epoch}")
    def after_epoch (self, epoch, val, **kwargs): print(f"Finished epoch {epoch} at {val}")

In [ ]:
cb_cls = PrintStepCallback()
train_n_sessions(cb=cb_cls)

Starting epoch 0
Finished epoch 0 at 0
Starting epoch 1
Finished epoch 1 at 1
Starting epoch 2
Finished epoch 2 at 5
Starting epoch 3
Finished epoch 3 at 14
Starting epoch 4
Finished epoch 4 at 30


30

In the above, kwargs was kept to make the code more future proof. We may later want to pass in more vars.

## Using callbacks to modify behavior

We can check if a callback class has certain callbacks and only do those callbacks if it does.

In [ ]:
def train_n_sessions(n=5, cb=None):
    res = 0
    for i in range(n):
        # Before epoch
        if cb and hasattr(cb,'before_epoch'): cb.before_epoch(i)
        calc = do_calc(i)
        res += calc
        # After epoch
        if cb and hasattr(cb,'after_epoch'):
            if cb.after_epoch(i, val=res):
                print('Stopping early')
                break
    return res

In [ ]:
class PrintAfterCallback():
    def after_epoch(self, epoch, val):
        print(f"After epoch: {epoch} | val: {val}")
        if val>10: return True

In [ ]:
cb_cls = PrintAfterCallback()
train_n_sessions(cb=cb_cls)

After epoch: 0 | val: 0
After epoch: 1 | val: 1
After epoch: 2 | val: 5
After epoch: 3 | val: 14
Stopping early


14

Let's change the way the calculation is being done! We'll do this by making train_n_sessions into a class and the value it calculates an attr of it.

In [ ]:
class TrainSessions():
    def __init__(self, cb=None, epochs=5):
        self.cb, self.res, self.epochs = cb, 0, epochs
        
    def callback(self, cb_name, *args):
        if not self.cb: return               # no callbacks → done.
        cb = getattr(self.cb, cb_name, None) # look in cb list for cb_name (a callable).
        if cb: return cb(self, *args)        # if not None: do callable w/ *args.
        
    def do_calc(self, x):
        sleep(1)
        return x*x
    
    def do_epochs(self):
        for i in range(self.epochs):
            self.callback('before_epoch', i)
            self.res += self.do_calc(i)
            if self.callback('after_epoch', i):
                print('Stopping early')
                break

In [ ]:
class ModifyingCallback():
    def after_epoch(self, trainer, epoch):
        print(f"After {epoch}: {trainer.res}")
        if trainer.res>10: return True
        if trainer.res<3: trainer.res = trainer.res*2

In [ ]:
trainer = TrainSessions(ModifyingCallback())

In [ ]:
trainer.do_epochs()

After 0: 0
After 1: 1
After 2: 6
After 3: 15
Stopping early


In [ ]:
trainer.res

15